In [ ]:
from nltk import download
download('punkt')
download('stopwords')

In [ ]:
from nltk.corpus import movie_reviews, stopwords
from string import punctuation

# remove stopwords and punctuation
def clean_words(words):
    return [w for w in words if w not in stopwords.words("english") and w not in punctuation]

In [ ]:
def get_word_dict(words):
    words = clean_words(words)
    return dict([(w, True) for w in words])

In [ ]:
import pickle

model_file = open("sa_classifier.pickle", "rb")
model = pickle.load(model_file)
model_file.close()

In [ ]:
from nltk.tokenize import word_tokenize

def get_sentiment(review):
    words = word_tokenize(review)
    words = clean_words(words)
    words = get_word_dict(words)
    return model.classify(words)

In [ ]:
positive_review = "This movie is amazing, with witty dialog and beautiful shots."
get_sentiment(positive_review)

In [ ]:
negative_review = "I hated everything about this unimaginitive mess. Two thumbs down!"
get_sentiment(negative_review)